In [ ]:
from Historic_Crypto import Cryptocurrencies, HistoricalData
import numpy as np
import datetime

## Find the BTC USD pair

In [ ]:
pairs = Cryptocurrencies().find_crypto_pairs()

In [ ]:
pairs[pairs['id'].str.contains('BTC') * pairs['id'].str.contains('USD')]

## Download Small Test Set

In [ ]:
btc_test = HistoricalData(
    ticker = "BTC-USD",
    granularity = (60 * 60 * 24), # 1 hour in seconds
    start_date = "2020-01-01-00-00",
    end_date = "2020-01-08-00-00"
).retrieve_data()

In [ ]:
btc_test.head()

Looks good, ready to download full desired date range.

## Download Desired Date Range

I am most intersted in bear markets since 2017 as I'm looking at best time to DCA buy during bear market conditions.

In [ ]:
raw_data =  HistoricalData(
        ticker = "BTC-USD",
        granularity = (60 * 60 * 24), # 1 hour in seconds
        start_date = "2017-01-01-00-00",
        end_date = "2022-06-25-00-00"
    ).retrieve_data();

## Process Data

In [ ]:
raw_data.info()

Looks like no missing values in any of the columns which is excellent.

In [ ]:
processed_data = raw_data.copy()

In [ ]:
test = processed_data.index.to_list()
test0 = test[0]

In [ ]:
test0.day_name()
test0.year

In [ ]:
processed_data.dtypes

In [ ]:
processed_data['avg'] = processed_data[['low','high','open','close']].mean(axis=1)
processed_data['1w_sma'] = processed_data['close'].rolling(window = 7 ).mean()
processed_data['20w_sma'] = processed_data['close'].rolling(window = 7 * 20).mean()
processed_data['year'] = [date.year for date in  processed_data.index.to_list()]
processed_data['market'] = np.where(processed_data['1w_sma'] < processed_data['20w_sma'], "bear","bull")
processed_data['day_of_week'] = [date.day_name() for date in  processed_data.index.to_list()]
processed_data['week_of_year'] = [date.weekofyear for date in  processed_data.index.to_list()]
processed_data['week_of_year'] = [date.weekofyear for date in  processed_data.index.to_list()]
processed_data['year_week_of_year'] = processed_data['year'].astype('str') + "_" + processed_data['week_of_year'].astype('str')

In [ ]:
processed_data[8:20]

In [ ]:
bear_data = processed_data[processed_data['market'] == 'bear'][['day_of_week','year_week_of_year','avg']]
bear_data.head(10)

In [ ]:
min_avg_per_week = bear_data.groupby('year_week_of_year').min('avg')
min_avg_per_week.rename(columns = {"avg":"min_weekly"},inplace=True)
min_avg_per_week.head()

In [ ]:
joined = bear_data.join(min_avg_per_week, on = 'year_week_of_year')
joined.head()

In [ ]:
joined['is_min'] = np.where(joined['min_weekly'] == joined['avg'], True, False)
joined.head(10)

In [ ]:
best_days_bear = joined[joined['is_min']]['day_of_week'].value_counts()

In [ ]:
best_days_bear